In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import time

In [3]:
data = pd.read_csv(r'/content/drive/MyDrive/Files.csv/video_subtitles.csv')
data.head()

,num,name,content_clean
0,9180533,the.message.(1976).eng.1cd,in the name of god the most gracious the...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,ah there s princess dawn and terry with the...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,yumi s cells 2 episode extremely polite y...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,yumi s cells 2 episode 39 laptop firs...
4,9180600,broker.(2022).eng.1cd,if you re going to throw it away then don...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41250 entries, 0 to 41249
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   num            41250 non-null  int64 
 1   name           41250 non-null  object
 2   content_clean  41250 non-null  object
dtypes: int64(1), object(2)
memory usage: 966.9+ KB


In [5]:
df = data[:5000]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   num            5000 non-null   int64 
 1   name           5000 non-null   object
 2   content_clean  5000 non-null   object
dtypes: int64(1), object(2)
memory usage: 117.3+ KB


In [6]:
import re

def preprocess_text(text):
    # Replace every two or more spaces with a single space
    cleaned_text = re.sub(r'\s{2,}', ' ', text)
    return cleaned_text
df['content_clean'] = df['content_clean'].apply(preprocess_text)

<ipython-input-6-04f61785ce7e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content_clean'] = df['content_clean'].apply(preprocess_text)


In [7]:
import tensorflow as tf

# Check GPU availability
if tf.test.gpu_device_name():
    print('GPU available')
else:
    print('GPU not available')

# Check TPU availability
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("TPU available")
except ValueError:
    print("TPU not available")

GPU not available
TPU not available


In [11]:
pip install semantic-text-splitter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 15.8 MB/s eta 0:00:00


In [8]:
from semantic_text_splitter import TextSplitter
from tokenizers import Tokenizer
MAX_TOKENS = 1500

# Load Hugging Face tokenizer
tokenizer = Tokenizer.from_pretrained("bert-base-uncased")

# Define TextSplitter with capacity (should match or be smaller than MAX_TOKENS)
splitter = TextSplitter.from_huggingface_tokenizer(tokenizer, capacity=MAX_TOKENS)

counter = 0

def chunk_document(content):
    global counter
    counter += 1
    start_time = time.time()

    # Perform the chunking operation
    chunks_with_model = splitter.chunks(content)  # ✅ Fixed

    end_time = time.time()
    execution_time = end_time - start_time
    print(counter)
    print("Chunking operation completed in {:.2f} seconds".format(execution_time))

    return chunks_with_model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
with tf.device('/device:GPU:0'):  # Or '/device:TPU:0' for TPU
# Apply chunking to each document in the DataFrame
    df['chunks'] = df['content_clean'].apply(chunk_document)

Streaming output truncated to the last 5000 lines.
532
Chunking operation completed in 0.11 seconds
533
Chunking operation completed in 0.11 seconds
534
Chunking operation completed in 0.12 seconds
535
Chunking operation completed in 0.11 seconds
536
Chunking operation completed in 0.12 seconds
537
Chunking operation completed in 0.11 seconds
538
Chunking operation completed in 0.13 seconds
539
Chunking operation completed in 0.11 seconds
540
Chunking operation completed in 0.12 seconds
541
Chunking operation completed in 0.11 seconds
542
Chunking operation completed in 0.06 seconds
543
Chunking operation completed in 0.12 seconds
544
Chunking operation completed in 0.11 seconds
545
Chunking operation completed in 0.12 seconds
546
Chunking operation completed in 0.09 seconds
547
Chunking operation completed in 0.12 seconds
548
Chunking operation completed in 0.12 seconds
549
Chunking operation completed in 0.06 seconds
550
Chunking operation completed in 0.13 seconds
551
Chunking opera

In [11]:
for i, chunk in enumerate(df['chunks'][0]):
    print(f"CHUNK {i+1}: ", chunk )

CHUNK 1:  in the name of god the most gracious the most merciful from muhammad the messenger of god to heraclius the emperor of byzantium greetings to him who is the follower of righteous guidance i bid you to hear the divine call i am the messenger of god to the people accept islam for your salvation he speaks of a new prophet in arabia was it like this when john the baptist came to king herod out of the desert crying about salvation to muqawqis patriarch of alexandria kisra emperor of persia muhammad calls you with the call of god accept islam for your salvation embrace islam you come out of the desert smelling of camel and goat to tell persia where he should kneel muhammad messenger of god who gave him this authority god sent muhammad as a mercy to mankind the scholars and historians of islam the university of al azhar in cairo the high islamic congress of the shiat in lebanon the makers of this film honour the islamic tradition which holds that the impersonation of the prophet offe

In [12]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
counter = 0
def embedding(content):
    global counter  # Use the global counter variable
    counter += 1  # Increment the counter
    start_time = time.time()
    # Perform your embedding operation
    query_embedding = model.encode(content)

    end_time = time.time()
    execution_time = end_time - start_time
    print(counter)

    print("Embedding operation completed in {:.2f} seconds".format(execution_time))
    return query_embedding

In [14]:
import tensorflow as tf

with tf.device('/device:GPU:0'):  # Or '/device:TPU:0' for TPU
    df['emb'] = df['chunks'].apply(embedding)

Streaming output truncated to the last 5000 lines.
2501
Embedding operation completed in 0.80 seconds
2502
Embedding operation completed in 0.82 seconds
2503
Embedding operation completed in 0.80 seconds
2504
Embedding operation completed in 1.00 seconds
2505
Embedding operation completed in 0.49 seconds
2506
Embedding operation completed in 0.48 seconds
2507
Embedding operation completed in 0.49 seconds
2508
Embedding operation completed in 0.48 seconds
2509
Embedding operation completed in 0.49 seconds
2510
Embedding operation completed in 0.48 seconds
2511
Embedding operation completed in 0.49 seconds
2512
Embedding operation completed in 0.72 seconds
2513
Embedding operation completed in 0.74 seconds
2514
Embedding operation completed in 0.72 seconds
2515
Embedding operation completed in 1.09 seconds
2516
Embedding operation completed in 1.29 seconds
2517
Embedding operation completed in 1.22 seconds
2518
Embedding operation completed in 1.57 seconds
2519
Embedding operation comple

<ipython-input-14-d862ce863e0e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['emb'] = df['chunks'].apply(embedding)


In [15]:
df.head()

,num,name,content_clean,chunks,emb
0,9180533,the.message.(1976).eng.1cd,in the name of god the most gracious the most...,[in the name of god the most gracious the most...,"[[-0.03874262, 0.1416285, -0.07002451, -0.0662..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,ah there s princess dawn and terry with the b...,[ah there s princess dawn and terry with the b...,"[[-0.07779901, -0.01926905, 0.04703944, -0.072..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,yumi s cells 2 episode extremely polite yumi ...,[yumi s cells 2 episode extremely polite yumi ...,"[[-0.14962061, -0.14706889, 0.05522108, -0.043..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,yumi s cells 2 episode 39 laptop first place ...,[yumi s cells 2 episode 39 laptop first place ...,"[[-0.117271274, -0.06049585, 0.06533918, -0.02..."
4,9180600,broker.(2022).eng.1cd,if you re going to throw it away then don t g...,[if you re going to throw it away then don t g...,"[[-0.065826476, 0.0031216121, 0.017494569, -0...."


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   num            5000 non-null   int64 
 1   name           5000 non-null   object
 2   content_clean  5000 non-null   object
 3   chunks         5000 non-null   object
 4   emb            5000 non-null   object
dtypes: int64(1), object(4)
memory usage: 195.4+ KB


In [19]:
print(df['emb'][5].shape)
print('*'*20)
print(len(df['emb'][0]))

(3, 384)
********************
7


In [20]:
df.to_csv('video_subtitles&emb.csv',index=False)

In [21]:
exploded_df = df.explode(['chunks', 'emb'])

In [22]:
exploded_df.shape

(22619, 5)

In [23]:
exploded_df.to_csv('full_video_subtitles.csv',index=False)

In [24]:
exploded_df = exploded_df.reset_index(drop=True)

In [25]:
exploded_df.shape

(22619, 5)

In [26]:
embeddings = exploded_df['emb'].tolist()
ids = exploded_df.index.astype(str).tolist()  # Convert integers to stringsids = cleaned_df.index.astype(str).tolist()

documents = exploded_df['chunks'].tolist()
metadata = exploded_df.drop(['content_clean','chunks','emb'], axis = 1).to_dict(orient = 'records')

In [27]:
documents[20000]

'father gave me fortitude when i was sick as a child he died soon after i married i m sorry if i alarmed you why do you do this sketch me it is my work and my habit you take care of people i draw them you have a husband at war no i m widowed it s been well quite a while now or do you have an amour forgive me i am too inquisitive painters are very aware of color for example your cheeks just turned pink it s complicated it s not a doctor oh no pauvre i once knew a doctor who could say precisely what i felt by the pulse of a single vein he sounds most observant he came to salpetriere to be a surgeon well they all come for that but more than the skill with his hands his greatest gift was his mind diagnostique seeing subtle signs of malady his treatment was very imaginative did you love him do you love yours miss mary i m sorry i ve been told to prepare you for travel travel no i i m to remain here major mcburney ordered it what no you you tell him i want to see him immediately i m not goin

In [28]:
embeddings[20000]

array([-1.08318366e-02,  1.14078997e-02,  5.36149070e-02, -8.29184521e-03,
        1.13422209e-02,  2.20190659e-02,  1.26442716e-01, -8.67980253e-03,
        5.33011835e-03, -7.39184245e-02, -6.24224804e-02, -1.85600426e-02,
        4.42798436e-02, -2.25373488e-02, -5.38465418e-02,  3.56787704e-02,
        1.58189535e-02,  1.67831481e-02, -9.34234709e-02,  7.03647435e-02,
       -3.92334536e-03,  7.99494758e-02, -1.36089213e-02, -6.37173802e-02,
       -9.04341340e-02,  2.03320514e-02, -4.13965061e-02, -2.92496681e-02,
       -4.26860265e-02, -4.52289265e-03,  3.01101618e-02,  2.75595319e-02,
        3.09035424e-02,  4.83868197e-02, -1.68869291e-02,  2.30934694e-02,
       -8.53673816e-02,  1.10261984e-01, -5.23687974e-02,  2.69253794e-02,
       -8.70508328e-03,  3.19940299e-02,  6.42745569e-02, -2.90993098e-02,
        4.41749394e-02, -8.30030441e-02, -4.56466302e-02, -2.34986693e-02,
        1.15208581e-01,  1.87002700e-02, -1.25130162e-01, -5.34768030e-02,
       -1.15279555e-02, -

In [29]:
embeddings_as_lists = [embedding.tolist() for embedding in embeddings]

In [31]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.1 MB/s eta 0:00:

In [32]:
import chromadb
import chromadb.config

from chromadb.config import Settings

client = chromadb.PersistentClient(path="/content")

In [33]:
collection = client.create_collection(name="Search_Engine",
                                      metadata={"hnsw:space": "cosine"} # l2 is the default
)

In [35]:
for i, embedding in enumerate(embeddings_as_lists):

    # Add the embeddings list to your collection
    collection.add(
            documents=documents[i],
            embeddings=embeddings_as_lists[i],
            ids=ids[i],
            metadatas=metadata[i]
        )

In [36]:
results = collection.query(
    query_texts=["up when he has fallen   is described by them as upsetting social order   to this inhumanity has come a man   whom god chose   and in that we believe   you ve overcome  i beg you to collect yourself   i speak of the messenger of god   muhammad teaches us to worship one god   to speak truth   to love our neighbors as ourselves   to give charity even a smile can be charity   to protect women from misuse   to shelter orphans   and to turn away from gods of wood and stone   i cannot keep still and hear this blasphemy   we are an ancient civilization   to call our gods wood and stone is to speak ignorantly of them   the idol the form is not what we worship   but the spirit that resides within the form   i agree that idolatry is not always fully understood   thank you   now let me bring him back to the women   god made woman to be the proper companion of man   she is different but equal   equal   we buy them   feed them  clothe them   use them  discard them   women equal to us   god created man from one male and one female   amr  you must respect in all woman the womb that bore you   why are your  guards so tongue tied   while this only guard is eloquent   god has spoken to us before   through abraham  noah  moses and through jesus christ   why should we be so surprised that god speaks to us now through muhammad   who taught you those names   they are named in the quran   i knew muhammad when he was an orphan minding sheep   and we knew christ as a carpenter   what christ says and what your muhammad says   is like two raised from the same land   they are lying to you they deny christ   you worship three gods  they say father  son and holy ghost  they say   what do you say of christ   they say god cannot have a son   christ is not the son of god   speak to me of christ   we say of christ what our prophet has taught us   that god cast his holy spirit into the womb of a virgin named  mary   and that she conceived christ  the apostle of god   the apostle he says not the son  not the son   what does your miracle  your quran say of the birth of our dear lord jesus christ   may i relate the words   come closer to me   in the name of god most gracious  most merciful   relate in the book  the story of mary   how she withdrew from her family to a place in the east   how we sent to her our angel  gabriel  who said   i am a messenger from your god   to announce the birth of a holy son to you   she said   how shall i  mary  have a son when no man has touched me   and gabriel replied   for your lord says  it will happen   we appoint him as a sign onto man   and a mercy from us"],
    n_results=5)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 38.6MiB/s]


In [37]:
print(results)

{'ids': [['1', '6', '2', '13332', '21961']], 'embeddings': None, 'documents': [['there jafar when god gave him these words dawn is coming up ammar you first then you jaafar ammar you kept your mother awake all night with worry i m sorry father where were you have you been with muhammad again what will happen now forgive him it was my fault i did it that god has helped us all our lives but it fell it could not even help itself what talk have you been listening to the real god is unseen he s not made of clay ammar we see the gods in the kaaba every day i m afraid for you you re listening to people who will hurt you i m listening to muhammad mother muhammad is generous yes he gives he shares he ll pass no man without a smile but he is spreading dangerous ideas dangerous ideas that no man should start that the rich should not afloat the poor the strong should not oppress the weak are these dangerous ideas girls should not be forced into marriage but be able to choose or refuse by only toni

In [38]:
# Connect to the ChromaDB client and collection
client = chromadb.PersistentClient(path="/content")

collection = client.get_collection("Search_Engine")

In [39]:
model = SentenceTransformer('all-MiniLM-L6-v2')
query_embedding = model.encode(["up when he has fallen   is described by them as upsetting social order   to this inhumanity has come a man   whom god chose   and in that we believe   you ve overcome  i beg you to collect yourself   i speak of the messenger of god   muhammad teaches us to worship one god   to speak truth   to love our neighbors as ourselves   to give charity even a smile can be charity   to protect women from misuse   to shelter orphans   and to turn away from gods of wood and stone   i cannot keep still and hear this blasphemy   we are an ancient civilization   to call our gods wood and stone is to speak ignorantly of them   the idol the form is not what we worship   but the spirit that resides within the form   i agree that idolatry is not always fully understood   thank you   now let me bring him back to the women   god made woman to be the proper companion of man   she is different but equal   equal   we buy them   feed them  clothe them   use them  discard them   women equal to us   god created man from one male and one female   amr  you must respect in all woman the womb that bore you   why are your  guards so tongue tied   while this only guard is eloquent   god has spoken to us before   through abraham  noah  moses and through jesus christ   why should we be so surprised that god speaks to us now through muhammad   who taught you those names   they are named in the quran   i knew muhammad when he was an orphan minding sheep   and we knew christ as a carpenter   what christ says and what your muhammad says   is like two raised from the same land   they are lying to you they deny christ   you worship three gods  they say father  son and holy ghost  they say   what do you say of christ   they say god cannot have a son   christ is not the son of god   speak to me of christ   we say of christ what our prophet has taught us   that god cast his holy spirit into the womb of a virgin named  mary   and that she conceived christ  the apostle of god   the apostle he says not the son  not the son   what does your miracle  your quran say of the birth of our dear lord jesus christ   may i relate the words   come closer to me   in the name of god most gracious  most merciful   relate in the book  the story of mary   how she withdrew from her family to a place in the east   how we sent to her our angel  gabriel  who said   i am a messenger from your god   to announce the birth of a holy son to you   she said   how shall i  mary  have a son when no man has touched me   and gabriel replied   for your lord says  it will happen   we appoint him as a sign onto man   and a mercy from us"])

In [40]:
results = collection.query(
                query_embeddings=query_embedding,
                n_results=5,
            )

In [41]:
print(results)

{'ids': [['1', '6', '2', '13332', '21961']], 'embeddings': None, 'documents': [['there jafar when god gave him these words dawn is coming up ammar you first then you jaafar ammar you kept your mother awake all night with worry i m sorry father where were you have you been with muhammad again what will happen now forgive him it was my fault i did it that god has helped us all our lives but it fell it could not even help itself what talk have you been listening to the real god is unseen he s not made of clay ammar we see the gods in the kaaba every day i m afraid for you you re listening to people who will hurt you i m listening to muhammad mother muhammad is generous yes he gives he shares he ll pass no man without a smile but he is spreading dangerous ideas dangerous ideas that no man should start that the rich should not afloat the poor the strong should not oppress the weak are these dangerous ideas girls should not be forced into marriage but be able to choose or refuse by only toni